In [13]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import numpy as np
import pandas as pd

In [19]:
# v0_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v0.csv')[['id', 'target']]
# v1_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v1.csv')[['id', 'target']]
# v2_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v2.csv')[['id', 'target']]
# v3_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v3.csv')[['id', 'target']]
# v4_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v4.csv')[['id', 'target']]
# v5_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v5.csv')[['id', 'target']]
# v6_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v6.csv')[['id', 'target']]
v13_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v13.csv')[['id', 'signal_depth', 'target']]
v14_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v14.csv')[['id', 'signal_depth', 'target']]
v15_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v15.csv')[['id', 'signal_depth', 'target']]
# v16_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v16.csv')[['id', 'signal_depth', 'target']]
# v17_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v17.csv')[['id', 'signal_depth', 'target']]
# train = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/train_labels.csv')
test = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
# train = train.loc[train['target'] != -1]
# v2_df = pd.read_csv('input/g2net-detecting-continuous-gravitational-waves/v2.csv')

In [20]:
# v0_df['path'] = v0_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v0/{x}.pickle')
# v1_df['path'] = v1_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v1/{x}.pickle')
# v2_df['path'] = v2_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v2/{x}.pickle')
# v3_df['path'] = v3_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v3/{x}.pickle')
# v4_df['path'] = v4_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v4/{x}.pickle')
# v5_df['path'] = v5_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v5/{x}.pickle')
# v6_df['path'] = v6_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v6/{x}.pickle')
v13_df['path'] = v13_df['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v13/{x}.pickle')
v14_df['path'] = v14_df['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v14/{x}.pickle')
v15_df['path'] = v15_df['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v15/{x}.pickle')
# v16_df['path'] = v16_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v16/{x}.pickle')
# v17_df['path'] = v17_df['id'].apply(
#     lambda x: f'input/g2net-detecting-continuous-gravitational-waves/v17/{x}.pickle')

In [21]:
v13_df['group'] = v13_df['id'].apply(lambda x: x.split('_')[0])
v14_df['group'] = v14_df['id'].apply(lambda x: x.split('_')[0])
v15_df['group'] = v15_df['id'].apply(lambda x: x.split('_')[0])

In [22]:
v13_df['id'] = v13_df['id'] + '_v13'
v14_df['id'] = v14_df['id'] + '_v14'
v15_df['id'] = v15_df['id'] + '_v15'

In [23]:
# # pd.concat([v0_df, v1_df, v2_df]).reset_index(drop=True).to_csv(
# pd.concat([v1_df, v4_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v1_v4.csv', index=False)

# pd.concat([v1_df, v4_df, v5_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v1_v4_v5.csv', index=False)

# pd.concat([v1_df, v4_df, v5_df, v6_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v1_v4_v5_v6.csv', index=False)


pd.concat([v13_df, v14_df, v15_df]).reset_index(drop=True).to_csv(
    'input/g2net-detecting-continuous-gravitational-waves/concat_v13_v14_v15.csv', index=False)

# pd.concat([v13_df, v14_df, v15_df, v16_df, v17_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v13_v14_v15_v16_v17.csv', index=False)

# pd.concat([v16_df, v17_df]).reset_index(drop=True).to_csv(
#     'input/g2net-detecting-continuous-gravitational-waves/concat_v16_v17.csv', index=False)

In [6]:
test['path'] = test['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/test/{x}.pickle')
test

,id,target,path
0,00054c878,0.5,input/g2net-detecting-continuous-gravitational...
1,0007285a3,0.5,input/g2net-detecting-continuous-gravitational...
2,00076c5a6,0.5,input/g2net-detecting-continuous-gravitational...
3,001349290,0.5,input/g2net-detecting-continuous-gravitational...
4,001a52e92,0.5,input/g2net-detecting-continuous-gravitational...
...,...,...,...
7970,ffbce04ef,0.5,input/g2net-detecting-continuous-gravitational...
7971,ffc2d976b,0.5,input/g2net-detecting-continuous-gravitational...
7972,ffc905909,0.5,input/g2net-detecting-continuous-gravitational...
7973,ffe276f3e,0.5,input/g2net-detecting-continuous-gravitational...


In [9]:
test.to_csv('input/g2net-detecting-continuous-gravitational-waves/test.csv', index=False)

In [10]:
train['path'] = train['id'].apply(
    lambda x: f'input/g2net-detecting-continuous-gravitational-waves/train/{x}.pickle')
train
train.to_csv('input/g2net-detecting-continuous-gravitational-waves/train.csv', index=False)